In [1]:
from parlai.tasks.md_gender import convai2
import parlai.tasks.md_gender.utils as md_utils

from copy import deepcopy


In [2]:
data_path = '/home/kuina/anaconda3/envs/parlai/lib/python3.10/site-packages/data/'

opt = {'datapath': data_path, #Donde están los datos
    'datatype':'train',
    'labels_to_use':'all',
    'add_unknown_classes': True, # help='Add unknown classes as neutral' # Esto es para incluir o no el about (???)
    'convai2_use_probably':True,
    'balance':False, # help='Whether to balance the data between classes during training',
    'task':'md_gender:convai2',
    #,'balance_valid':False, #help='Whether to balance the validation data'
    'unknown_temp':1.0 #help='Rate at which to sample examples from the unknown class',
}

In [3]:
convai = convai2.Convai2Teacher(opt)


[ Building data ... ]
11:39:20 | loading fbdialog data: /home/kuina/anaconda3/envs/parlai/lib/python3.10/site-packages/data/ConvAI2/train_both_original.txt
Missing cnt: 52 / 788640
Totals for self:
	unknown: 86711 (0.6596952244733378)
	female: 22503 (0.17120228847924163)
	male: 22227 (0.1691024870474205)
Totals for partner:
	unknown: 86472 (0.657876918161)
	female: 22345 (0.17000022823928607)
	male: 22624 (0.17212285359971394)


In [4]:
convai.data[0]

{'text': 'how many do you have ?',
 'episode_done': True,
 'your_persona': 'my hair is short.\nmy favorite color is yellow.\nmy boyfriend is in the army.\ni play piano.',
 'partner_persona': 'i love chickens.\ni work on a farm.\ni was born in kansas.\ni know how to drive a truck.',
 'id': 'ConvAI2 Gender',
 'labels': ['SELF:female'],
 'class_type': 'self'}

In [5]:
md_utils.get_data_stats(convai.data,key='labels',lst=True)

Total dataset counts:
ABOUT:female: 4428
ABOUT:gender-neutral: 116188
ABOUT:male: 10822
PARTNER:female: 22345
PARTNER:male: 22624
PARTNER:unknown: 86472
SELF:female: 22503
SELF:male: 22227
SELF:unknown: 86711


In [6]:
# data = {}
# for item in convai.data:
#     text = item['text']
#     labels = item['labels'][0]
#     if text in data and labels not in data[text]['labels']:
#         data[text]['labels'].append(labels)
#     else:
#         data[text] = {'labels':[labels]}

In [7]:
data = {}
for item in convai.data:
    text = item['text']
    labels = item['labels'][0]
    if text in data and labels not in data[text]:
        data[text].append(labels)
    else:
        data[text] = [labels]

In [8]:
data #No todos los elementos tendrán las 3 dimensiones

{'how many do you have ?': ['ABOUT:gender-neutral'],
 'being married is nice i bet , but i have friendly parakeet': ['PARTNER:male',
  'SELF:unknown',
  'ABOUT:gender-neutral'],
 "oh that is a bummer , i'm playing some nintendo games . i'm off today": ['SELF:unknown',
  'ABOUT:gender-neutral',
  'PARTNER:unknown'],
 'i need to study up if i want to stay in the dorm next year .': ['SELF:unknown'],
 'thank you too . i am from maine . you ?': ['SELF:female', 'PARTNER:unknown'],
 'oh yeah that creole charm': ['SELF:unknown',
  'ABOUT:gender-neutral',
  'PARTNER:unknown'],
 'what type of significant other are you looking for ? i like men that pretty tall .': ['SELF:female',
  'ABOUT:gender-neutral',
  'PARTNER:unknown'],
 "oh , i'm sorry . want to come play tennis with my two sisters and me ?": ['ABOUT:gender-neutral'],
 'do you like italian food ? i love mcdonald french fries .': ['PARTNER:unknown',
  'SELF:unknown'],
 'i do , i prefer to wear dresses , not mini skirts . i like to dress mo

In [9]:
print(len(data))

115877


In [10]:
# test_data = deepcopy(data)
# key_revisar = []
# key_eliminados = []
# for element in list(test_data):
#     labels = test_data[element]['labels']
#     if len(labels) < 3:
#         key_eliminados.append(element)
#         test_data.pop(element,'None')
#     elif len(labels) > 3:
#         key_revisar.append(element)
#         test_data.pop(element,'None')

In [11]:
test_data = deepcopy(data)
key_revisar = []
key_eliminados = []
for element in list(test_data):
    labels = test_data[element]
    if len(test_data[element]) < 3:
        key_eliminados.append(element)
        test_data.pop(element,'None')
    elif len(test_data[element]) > 3:
        key_revisar.append(element)
        test_data.pop(element,'None')

In [12]:
print(f"Elementos con 3 etiquetas: {len(test_data)}")
print(f"Elementos con menos de 3 etiquetas: {len(key_eliminados)}")
print(f"Elementos con más de 3 etiquetas: {len(key_revisar)}")

Elementos con 3 etiquetas: 48917
Elementos con menos de 3 etiquetas: 66892
Elementos con más de 3 etiquetas: 68


In [13]:
test_data

{'being married is nice i bet , but i have friendly parakeet': ['PARTNER:male',
  'SELF:unknown',
  'ABOUT:gender-neutral'],
 "oh that is a bummer , i'm playing some nintendo games . i'm off today": ['SELF:unknown',
  'ABOUT:gender-neutral',
  'PARTNER:unknown'],
 'oh yeah that creole charm': ['SELF:unknown',
  'ABOUT:gender-neutral',
  'PARTNER:unknown'],
 'what type of significant other are you looking for ? i like men that pretty tall .': ['SELF:female',
  'ABOUT:gender-neutral',
  'PARTNER:unknown'],
 'a bit older but i respect it , i often read autobiographies when i am sleepy .': ['PARTNER:unknown',
  'ABOUT:gender-neutral',
  'SELF:unknown'],
 'like dragon ball z ? i like them all': ['PARTNER:male',
  'ABOUT:male',
  'SELF:unknown'],
 "no mom won't let us have any . distraction": ['ABOUT:gender-neutral',
  'PARTNER:female',
  'SELF:unknown'],
 'do you eat what you catch ? i do .': ['SELF:female',
  'ABOUT:gender-neutral',
  'PARTNER:male'],
 '__SILENCE__': ['ABOUT:gender-neutral

In [15]:
import json
with open('convai_all_labels_revised.json','w') as f:
    json.dump(test_data,f)